# MIDI hacking with mido
## Alesis QS6 programming

I'll use mido to write live sysex commands. The program edit format for Alesis QS6 is determined by bytes formed from bits as follows:

    F0 00 00 0E 0E 10 <0mmfffff> <0ssppppp> <0ccccddv> <0vvvvvvv> F7

    <mm>       = 0=Global, 1=Mix, 2=Program, 3=Effects
    <fffff>    = Function number 0 through 16, depending on mode
    <ss>       = Sound 1-4 (0-3) when <mm>=2, effect bus 1-4 (0-3) when <mm>=3
    <ppppp>    = Page 0 through 23, depending on mode and function
    <cccc>     = Channel 1 through 16 (0-15); Ignored unless in mix mode & <mm>=1 or 2
    <dd>       = Data entry pot number 1-4 (0-3)
    <vvvvvvvv> = Parameter value, 8 bit 2’s complement

All parameters to be edited must be sent in this format (12 MIDI bytes), regardless of the number of bits required to transmit the value of the parameter.  When the QS receives this message, it will edit the specified parameter to the new value and display it.  If the function and page selected does not exist in the current configuration, the command will cause the nearest legal function, page, and parameter to be selected, but no edit will occur.  If a Mix edit is sent while in program mode, it will be ignored.  If this command is received while in compare, it will be ignored.  If a program edit command is received, it will place the QS in Edit mode.  If the value received is out of range for the parameter selected, the range will be limited to the nearest legal value.  The function and page numbers for each parameter are shown in the next section.


In [1]:
import mido

In [12]:
msg = mido.Message('sysex')

In [230]:
# Set 
msg.data = [0x00,0x00,0x0E,0x0E,0x10]  # The first portion says: do a program parameter edit
msg.data += [0b01000001,  # 0mm is 010 for program edit, fffff is 00001 for function 1 (level)
             0b00000000,  # 0ss is 00 for sound 1, ppppp is 00000 for page 0
             0b00000000,  # 0cccc is 0000, ignored in pgm mode, dd is 00 for data0, 
             0b00000001]  # vvvvvvvv is 00000001 for 1

In [231]:
msg.hex()

'F0 00 00 0E 0E 10 41 00 10 01 F7'

In [8]:
outport = mido.open_output("Alesis QS6")

In [9]:
outport.send(msg)

In [276]:
# Set 
msg.data = [0x00,0x00,0x0E,0x0E,0x10]  # The first portion says: do a program parameter edit
msg.data += [0b01001110,  # 0mm is 010 for program edit, fffff is 01110 for AENV
             0b00000000,  # 0ss is 00 for sound 1, ppppp is 00000 for page 0
             0b00000000,  # 0cccc is 0000, ignored in pgm mode, dd is 00 for data0, last bit is v!
             89]  # vvvvvvvv is 00000001 for 1

In [ ]:
aenv_

In [277]:
outport.send(msg)

    fffff Functions (page 0)
    00000  0 = Sound Grp (p3 Voice)
    00001  1 = Sound Volume (p1 Level)
    00010  2 = Effect Level (p4 Level)
    00011  3 = Semitone (p1 Pitch)
    00100  4 = Filter Freq (p1 Filter)
    00101  5 = Vel Curv (p1 amp/range)
    00110  6 = LO lim (p4 amp/range)
    00111  7 = MOD1SRC (p1 MOD1)
    01000  8 = Name (p1 Name)
    01001  9 = PLFO (p1 Pitch LFO)
    01010  10 = FLFO (p1 Filter LFO)
    01011  11 = ALFO (p1 Amp LFO)
    01100  12 = PENV Attack (p1 pitch env)
    01101  13 = FENV Attack (p1 filter env)
    01110  14 = AENV Attack (p1 amp env)
    01111  15 = TrackIN (p1 Track gen)
    10000  16 = Sound Type (p2 Voice)
    
Map to program table is located on page 14 of the SysEx guide.

I'm most interested in tweaking the envelopes in real time, so starting with AENV:

                                       Func Pg  Pot Off Lim bit bit address
    98.     Sound amp env attack        14  0   0   0   99  7   68:2-67:4
    99.     Sound amp env decay         14  0   1   0   99  7   69:1-68:3
    100.    Sound amp env sustain       14  0   2   0   99  7   70:0-69:2
    101.    Sound amp env release       14  0   3   0   99  7   70:7-70:1
    102.    Sound amp env delay         14  1   0   0   100 7   71:6-71:0
    103.    Sound amp env sustain decay 14  1   1   0   99  7   72:5-71:7
    104.    Sound amp env trig type     14  1   3   0   3   2   72:7-72:6
    105.    Sound amp env time track    14  2   0   0   1   1   73:0
    106.    Sound amp env sustain pedal 14  2   1   0   1   1   73:1
    107.    Sound amp env level         14  2   2   0   99  7   74:0-73:2



In [295]:
alesis_pgm_edit = [0x00,0x00,0x0E,0x0E,0x10]
aenv_attack = alesis_pgm_edit + [0b01001110,  # 0mm is 010 for program edit, fffff is 01110 for AENV
                                 0b00000000,  # 0ss is 00 for sound 1, ppppp is 00000 for page 0
                                 0b00000000] + [50] # 0cccc is 0000, ignored in pgm mode, dd is 00 for pot0
aenv_decay = alesis_pgm_edit + [0b01001110,  # 0mm is 010 for program edit, fffff is 01110 for AENV
                                0b00000000,  # 0ss is 00 for sound 1, ppppp is 00000 for page 0
                                0b00000010] + [50] # 0cccc is 0000, ignored in pgm mode, dd is 01 for pot1
aenv_sustain = alesis_pgm_edit + [0b01001110,  # 0mm is 010 for program edit, fffff is 01110 for AENV
                                0b00000000,  # 0ss is 00 for sound 1, ppppp is 00000 for page 0
                                0b00000100] + [50] # 0cccc is 0000, ignored in pgm mode, dd is 10 for pot2
aenv_release = alesis_pgm_edit + [0b01001110,  # 0mm is 010 for program edit, fffff is 01110 for AENV
                                0b00000000,  # 0ss is 00 for sound 1, ppppp is 00000 for page 0
                                0b00000110] + [50] # 0cccc is 0000, ignored in pgm mode, dd is 11 for pot3
aenv_delay = alesis_pgm_edit + [0b01001110,  # 0mm is 010 for program edit, fffff is 01110 for AENV
                                0b00000001,  # 0ss is 00 for sound 1, ppppp is 00001 for page 1
                                0b00000000] + [50] # 0cccc is 0000, ignored in pgm mode, dd is 00 for pot0
aenv_susdecay = alesis_pgm_edit + [0b01001110,  # 0mm is 010 for program edit, fffff is 01110 for AENV
                                0b00000001,  # 0ss is 00 for sound 1, ppppp is 00001 for page 1
                                0b00000010] + [50] # 0cccc is 0000, ignored in pgm mode, dd is 01 for pot1

In [302]:
msg.data = aenv_sustain
msg.hex()

'F0 00 00 0E 0E 10 4E 00 04 32 F7'

In [303]:
outport.send(msg)